In [6]:
from pesq import pesq
import librosa
import numpy as np
import torch

# SIR (Signal to Interference Ratio)


```MATLAB
% Copyright 2008 Emmanuel Vincent
% This software is distributed under the terms of the GNU Public License
% version 3 (http://www.gnu.org/licenses/gpl.txt)
% If you find it useful, please cite the following reference:
% Emmanuel Vincent, C�dric F�votte and R�mi Gribonval, "Performance
% measurement in blind audio source separation," IEEE Trans. on Audio,
% Speech and Language Processing, 14(4):1462-1469, 2006.
```

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=1643671

In [39]:
estim=[]

target=[]
for i in range(4) : 
    tmp_target, _ = librosa.load("target_"+str(i)+".wav",sr=16000)
    tmp_estim, _  = librosa.load("39_"+str(i)+".wav",sr=16000)
    
    print("target {} : {}".format(i,tmp_target.shape))

    target.append(tmp_target)
    estim.append(tmp_estim)

s_target = []
e_interf = []
    
for i in range(4) : 
    s_target.append(np.inner(estim[i],target[i])*target[i]/np.inner(target[i],target[i]))
    print("s_target[{}] : {}".format(i,s_target[i].shape))
    
    tmp = None
    for j in range(4) : 
        if i == j :
            continue
        if tmp is None : 
            tmp = np.inner(estim[i],target[j])*target[j]/np.inner(target[j],target[j])
        else : 
            tmp += np.inner(estim[i],target[idx_])*target[j]/np.inner(target[j],target[j])
    e_interf.append(tmp)


for i in range(4) : 
    print(i)
    print(s_target[i].shape)
    print(e_interf[i].shape)
    SIR = 10*np.log10((np.inner(s_target[i],s_target[i]))/np.inner(e_interf[i],e_interf[i]))
    print("SIR : {}".format(SIR))

    print("PESQ(wb) : {}".format(pesq(16000, target[i], estim[i], 'wb')))
    print("PESQ(nb) : {}".format(pesq(16000, target[i], estim[i], 'nb')))
    

target 0 : (16000,)
target 1 : (16000,)
target 2 : (16000,)
target 3 : (16000,)
365.9735
s_target[0] : (16000,)
533.4289
s_target[1] : (16000,)
301.60977
s_target[2] : (16000,)
507.89514
s_target[3] : (16000,)
0
(16000,)
(16000,)
SIR : 19.378488063812256
PESQ(wb) : 1.2397656440734863
PESQ(nb) : 1.6209020614624023
1
(16000,)
(16000,)
SIR : 18.61210823059082
PESQ(wb) : 2.105191469192505
PESQ(nb) : 2.6784563064575195
2
(16000,)
(16000,)
SIR : 9.921001195907593
PESQ(wb) : 1.8335928916931152
PESQ(nb) : 2.3431551456451416
3
(16000,)
(16000,)
SIR : 15.172652006149292
PESQ(wb) : 1.7513091564178467
PESQ(nb) : 1.973534345626831


bss_eval_toolkit이랑 SIR은 다른데 MATLAB 코드에서는 window 별로 projection 해서 그렇게 되는것 같다. 
다만 경향성은 동일하기 때문에 괜찮지 않을까? 

https://gitlab.inria.fr/bass-db/bss_eval/-/blob/master/v3.0/bss_eval_sources.m

(np.dot(a, b) / np.dot(b, b)) * b

## torch : SIR & PESQ for Batch Multi-target Multi-channel  

clean = [B,C,T]  
estim = [B,C,T]  

In [10]:
import torch
import torchaudio

16000 -> 2,3,16000  
Batch 연산보다 걍 for문 2개 돌리는게 지금은 편하다. dim 조절을 좀 해야해서 일단은 간단하게

In [134]:
def SIR(estim,target) :
    if estim.shape != target.shape : 
        raise Exception("ERROR::metric.py::SIR:: output shape != target shape | {} != {}".format(output.shape,target.shape))

    if len(estim.shape) != 2 : 
        raise Exception("ERROR::metric.py::SIR:: output dim {} != 2".format(len(output.shape)))
    #print("target : {}".format(target.shape))
    #print("estim : {}".format(estim.shape))
    n_target  = estim.shape[0]
    #print(n_target)
    
    s_target = []
    e_interf = []
    
   
    for i in range(n_target) : 
        s_target.append(torch.inner(estim[i],target[i])*target[i]/torch.inner(target[i],target[i]))
        #print("s_target[{}] : {}".format(i,s_target[i].shape))
        
        #print(torch.inner(estim[i],target[i]))

        tmp = None
        for j in range(n_target) : 
            if i == j :
                continue
            if tmp is None : 
                tmp = torch.inner(estim[i],target[j])*target[j]/torch.inner(target[j],target[j])
            else : 
                tmp += torch.inner(estim[i],target[j])*target[j]/torch.inner(target[j],target[j])
        e_interf.append(tmp)
    
    SIR =  torch.tensor(0.0)
    for i in range(n_target) : 
        t_SIR = (torch.inner(s_target[i],s_target[i]))/torch.inner(e_interf[i],e_interf[i])
        print("SIR : {}".format(t_SIR))
        SIR += t_SIR
    return 10*torch.log10(SIR)

In [135]:
target = []
output = []
for i in range(4) : 
    tmp_target, _ = librosa.load("target_"+str(i)+".wav",sr=16000)
    tmp_output, _  = librosa.load("39_"+str(i)+".wav",sr=16000)
    
    target.append(tmp_target)
    output.append(tmp_output)
    
target = torch.from_numpy(np.array(target))
output = torch.from_numpy(np.array(output))

# Exapnd dim to [B,N,C,T]
target = target.repeat(2,3,1,1).permute(0,2,1,3)
output = output.repeat(2,3,1,1).permute(0,2,1,3)

# Add noise
output[0:1,:,0:1,:] += (torch.rand(1,target.shape[1],1,target.shape[3])*2-1)
output[0:1,:,1:2,:] += target[0:1,:,2:3,:]

print(target.shape)
print(output.shape)
"""
    output : wav[n_target,n_sample]
    target : wav[n_target,n_sample]
"""

for b in range(2) : 
    for c in range(3) : 
        ret = SIR(output[b,:,c,:],target[b,:,c,:])
        print(ret)

torch.Size([2, 4, 3, 16000])
torch.Size([2, 4, 3, 16000])
SIR : 48.117103576660156
SIR : 40.32878112792969
SIR : 7.798915386199951
SIR : 29.698978424072266
tensor(21.0018)
SIR : 534.4221801757812
SIR : 288.35247802734375
SIR : 55.7760009765625
SIR : 148.43760681152344
tensor(30.1157)
SIR : 86.66600036621094
SIR : 72.64588165283203
SIR : 9.819747924804688
SIR : 32.90522003173828
tensor(23.0543)
SIR : 86.66600036621094
SIR : 72.64588165283203
SIR : 9.819747924804688
SIR : 32.90522003173828
tensor(23.0543)
SIR : 86.66600036621094
SIR : 72.64588165283203
SIR : 9.819747924804688
SIR : 32.90522003173828
tensor(23.0543)
SIR : 86.66600036621094
SIR : 72.64588165283203
SIR : 9.819747924804688
SIR : 32.90522003173828
tensor(23.0543)


In [159]:
import pesq

In [152]:
for j in range(32) :
    for i in range(4) : 
        #print(i)
        wb = pesq(16000,output[0,i,0,:].detach().numpy(),target[0,i,0,:].detach().numpy(),"wb")
        nb = pesq(16000,output[0,i,0,:].detach().numpy(),target[0,i,0,:].detach().numpy(),"nb")
        #print("{} {} {}".format(wb,nb,(wb+nb)/2))

# PESQ Exeception

In [163]:
 wb = pesq.pesq(16000,np.zeros(16000,np.float32),np.random.rand(16000),"wb",on_error=pesq.PesqError.RETURN_VALUES)

-7

In [1]:
import json

path_json="/home/data2/kbh/LGE/DESED_eval_simu_v5/0.json"

In [2]:
f = open(path_json,'r')
j = json.load(f)

In [4]:
j["n_src"]

1

In [6]:
import glob,os

In [7]:
xxx = [x for x in glob.glob(os.path.join("/home/data/kbh/CHiME4/isolated_ext/","tr*","*.CH1.Clean.wav"))]

In [10]:
xxx[-1]

'/home/data/kbh/CHiME4/isolated_ext/tr05_caf_simu/40M_40MO0301_CAF.CH1.Clean.wav'

In [13]:
aa = [1,2,3]
bb = [4,5,6]
aa+=bb
aa

[1, 2, 3, 4, 5, 6]